In [1]:
# List of imports
import geopandas as gpd
import pandas as pd
import os
import pandana as pnd

import matplotlib.pyplot as plt


In [37]:
# This GIST provides a template for conveting shapefiles into pandana networks. 
# Under a MIT Open Source License. https://opensource.org/licenses/MIT
# Researchers cite as Wasserman, D. Geopandas to Pandana Network Converter. (2019) GitHub repository, https://gist.github.com/d-wasserman/2d50671b37ee46b088e155293399a90c


def create_dist_column(networkGeoDataFrame):
    """Calculate the length to every edge from the givengeodataframe.
    networkGeoDataFrame:geoDataFrame Object with Linestrings
    """
    if(networkGeoDataFrame.crs['init'] == 'epsg:4326'):
        print('Converting to PT-TM06 for Portugal Projection')
        networkGeoDataFrame = networkGeoDataFrame.to_crs({'init':'epsg:3763'})
    print('Calculating length to Edges')
    networkGeoDataFrame['dist'] = networkGeoDataFrame['geometry'].length
    networkGeoDataFrame = networkGeoDataFrame.to_crs({'init': 'epsg:4326'})
    return networkGeoDataFrame

def get_nodes_and_edges(shp_file,rounding=5):
    """Use geopandas to read line shapefile and compile all paths and nodes in a line file based on a rounding tolerance.
    shp_file:path to polyline file with end to end connectivity
    rounding: tolerance parameter for coordinate precision"""
    edges = gpd.read_file(shp_file) 
#     edges = create_dist_column(edges)
    edges["from_x"]=edges["geometry"].apply(lambda x:round(x.coords[0][0],rounding))
    edges["from_y"]=edges["geometry"].apply(lambda x:round(x.coords[0][1],rounding))
    edges["to_x"]=edges["geometry"].apply(lambda x:round(x.coords[-1][0],rounding))
    edges["to_y"]=edges["geometry"].apply(lambda x:round(x.coords[-1][1],rounding))
    nodes_from = edges[["from_x","from_y"]].rename(index=str,columns={"from_x":"x","from_y":"y"})
    nodes_to = edges[["to_x","to_y"]].rename(index=str,columns={"to_x":"x","to_y":"y"})
    nodes = pd.concat([nodes_from,nodes_to],axis=0)
    nodes["xy"] = list(zip(nodes["x"], nodes["y"]))
    nodes = pd.DataFrame(nodes["xy"].unique(),columns=["xy"])
    nodes["x"] = nodes["xy"].apply(lambda x: x[0])
    nodes["y"] = nodes["xy"].apply(lambda x: x[1])
    nodes = nodes[["x","y"]].copy()
    return nodes , edges

def generate_pandana_store_from_shp(hdf5_path,shp_file,weights=["weight"],oneway=None,overwrite_existing=True,rounding=6):
    """Generate a pandana ready HDF5 store using geopandas (gdal required) and pandas. Python 3.5.
    hdf5_path(str): output path of HDF5 store holding two dataframes ["nodes","edges"]
    shp_file(str): input file that geopandas reads to make a graph based on end to end connectivity
    weights(list): weights columns transfered to the store edges. Name is maintained.
    oneway(str): series name where oneway streets (edges) are denoted with a 1, 0 denotes twoway. None, assumes
    twoway edge.
    overwrite_existing(bool): if true, the existing store is overwritten.
    rounding(int): the number of digits to round line coordinates to get unique nodes (precision)
    returns hdf5_path(str)"""
    if os.path.exists(hdf5_path):
        if overwrite_existing:
            print("Overwriting existing store...")
            os.remove(hdf5_path)
        else:
            print("Existing store at path: {0}".format(hdf5_path))
            return hdf5_path
    all_edges_twoway = True
    oneway_field_list = []
    if oneway is not None:
        all_edges_twoway = False
        oneway_field_list.append(oneway)
    print("Reading shapefile with geopandas: {0}...".format(shp_file))
    nodes, edges =get_nodes_and_edges(shp_file,rounding)
    h5store = pd.HDFStore(hdf5_path)
    print("Establishing node store...")
    df_nodes = nodes
    df_nodes["id"] = df_nodes.index.values
    df_nodes.index.rename("id",True)
    h5store['nodes'] = df_nodes
    edge_cnt = len(edges)
    print("Establishing edge store for {0} edges...".format(edge_cnt))
    df_edges= edges[['from_x','from_y','to_x','to_y'] + weights + oneway_field_list].copy()
    print("Transferring nodeids to edges...")
    df_edges=pd.merge(df_edges, df_nodes, how='left', left_on=['from_x','from_y'], right_on=['x','y'])
    df_edges=pd.merge(df_edges, df_nodes, how='left', left_on=['to_x','to_y'], right_on=['x','y'], suffixes=('_from', '_to'))
    #nodeids are duplicated on from the joined nodes, joined first to from, suffix to on next set
    df_edges.rename(columns= {'id_from': 'from', 'id_to': 'to'}, inplace=True)
    df_edges=df_edges[['from','to'] + weights + oneway_field_list]
    if all_edges_twoway:
        print("""Note: Edges are duplicated in this step, do not use the 'twoway' setting in the pandana network if using this 
             function""")
        twoway_edges = df_edges.copy()
        twoway_to = twoway_edges["to"].copy()
        twoway_edges["to"] = twoway_edges["from"]
        twoway_edges["from"] = twoway_to
        df_edges = pd.concat([df_edges,twoway_edges])
    else:
        print("Setting up edges based on oneway field...")
        twoway_edges = df_edges[df_edges[oneway]==0].copy()
        twoway_to = twoway_edges["to"].copy()
        twoway_edges["to"] = twoway_edges["from"]
        twoway_edges["from"] = twoway_to
        df_edges = pd.concat([df_edges,twoway_edges])
    h5store['edges']=df_edges
    h5store.close()
    print("Graph store construction complete...")
    return hdf5_path

In [4]:
# Load Shp to import
nodes, edges = get_nodes_and_edges('./data/Novo/Proj_Estradas.shp')

In [16]:
path = generate_pandana_store_from_shp('./data/portugal', './data/Novo/Proj_Estradas.shp', weights=['dist'])

Overwriting existing store...
Reading shapefile with geopandas: ./data/Novo/Proj_Estradas.shp...
Establishing node store...
Establishing edge store for 898234 edges...
Transferring nodeids to edges...
Note: Edges are duplicated in this step, do not use the 'twoway' setting in the pandana network if using this 
             function
Graph store construction complete...


In [22]:
myNetwork_raw = gpd.read_file('./portugal/gis_osm_roads_free_1.shp')

In [23]:
network_dist = gpd.read_file( './data/Novo/Proj_Estradas.shp')

In [24]:
myNetwork_raw.crs

{'init': 'epsg:4326'}

In [25]:
network_dist.crs

{'init': 'epsg:3763'}

In [28]:
myNetwork_raw.head()

,osm_id,code,fclass,name,ref,oneway,maxspeed,layer,bridge,tunnel,geometry
0,2954258,5122,residential,Rua da Lapa,None,B,0,0,F,F,"LINESTRING (-9.3979195 38.9544271, -9.39845200..."
1,3979151,5122,residential,Rua dos Pardais,None,B,0,0,F,F,"LINESTRING (-9.395928700000001 38.9566218, -9...."
2,3979152,5113,primary,None,EN 247,B,50,0,F,F,"LINESTRING (-9.419475500000001 38.9721483, -9...."
3,3979153,5122,residential,Travessa da Escola,None,B,0,0,F,F,"LINESTRING (-9.3994056 38.9551873, -9.39906080..."
4,3979154,5122,residential,Rua de Baixo,None,B,0,0,F,F,"LINESTRING (-9.3985764 38.9569177, -9.3985141 ..."


In [27]:
network_dist.head()

,osm_id,code,fclass,name,ref,oneway,maxspeed,layer,bridge,tunnel,dist,geometry
0,2954258,5122,residential,Rua da Lapa,None,B,0,0.0,F,F,332.0080,LINESTRING (-109638.1802253095 -78489.68064954...
1,3979151,5122,residential,Rua dos Pardais,None,B,0,0.0,F,F,377.9170,LINESTRING (-109462.2279786991 -78248.41792395...
2,3979152,5113,primary,None,EN 247,B,50,0.0,F,F,223.9630,LINESTRING (-111479.0138039726 -76496.10140458...
3,3979153,5122,residential,Travessa da Escola,None,B,0,0.0,F,F,70.1734,LINESTRING (-109765.8320163768 -78403.49371099...
4,3979154,5122,residential,Rua de Baixo,None,B,0,0.0,F,F,242.9640,LINESTRING (-109691.284256143 -78212.382669349...


In [29]:
mergedDf = pd.merge(myNetwork_raw, network_dist[['dist']], left_on=myNetwork_raw.osm_id, right_on=network_dist.osm_id)

In [31]:
mergedDf = mergedDf.drop(columns='key_0')
mergedDf

,osm_id,code,fclass,name,ref,oneway,maxspeed,layer,bridge,tunnel,geometry,dist
0,2954258,5122,residential,Rua da Lapa,None,B,0,0,F,F,"LINESTRING (-9.3979195 38.9544271, -9.39845200...",332.00800
1,3979151,5122,residential,Rua dos Pardais,None,B,0,0,F,F,"LINESTRING (-9.395928700000001 38.9566218, -9....",377.91700
2,3979152,5113,primary,None,EN 247,B,50,0,F,F,"LINESTRING (-9.419475500000001 38.9721483, -9....",223.96300
3,3979153,5122,residential,Travessa da Escola,None,B,0,0,F,F,"LINESTRING (-9.3994056 38.9551873, -9.39906080...",70.17340
4,3979154,5122,residential,Rua de Baixo,None,B,0,0,F,F,"LINESTRING (-9.3985764 38.9569177, -9.3985141 ...",242.96400
5,3979155,5155,steps,Rua da Quinta,None,B,0,0,F,F,"LINESTRING (-9.398498500000001 38.9555472, -9....",6.96085
6,3979156,5122,residential,Travessa da Eira,None,B,0,0,F,F,"LINESTRING (-9.400195999999999 38.9559047, -9....",70.16490
7,3979157,5122,residential,Rua do Coxo,None,B,0,0,F,F,"LINESTRING (-9.399060800000001 38.9543998, -9....",284.03100
8,3979159,5122,residential,Travessa do Cotovelo,None,B,0,0,F,F,"LINESTRING (-9.3991825 38.9559408, -9.39907479...",67.73940
9,3979160,5113,primary,None,EN 247,F,0,0,F,F,"LINESTRING (-9.4195262 38.9723667, -9.4194897 ...",68.68450


In [32]:
mergedDf.to_excel('./data/networkNoCSR.xlsx')

KeyboardInterrupt: 

In [33]:
mergedDf.to_file('./data/networkNoCSR.shp')

In [38]:
path = generate_pandana_store_from_shp('./data/portugal', './data/networkNoCSR.shp', weights=['dist'])

Reading shapefile with geopandas: ./data/networkNoCSR.shp...
Establishing node store...
Establishing edge store for 895328 edges...
Transferring nodeids to edges...
Note: Edges are duplicated in this step, do not use the 'twoway' setting in the pandana network if using this 
             function
Graph store construction complete...


In [34]:
test = gpd.read_file('./data/networkNoCSR.shp')

In [35]:
test.head()

,osm_id,code,fclass,name,ref,oneway,maxspeed,layer,bridge,tunnel,dist,geometry
0,2954258,5122,residential,Rua da Lapa,None,B,0,0,F,F,332.0080,"LINESTRING (-9.3979195 38.9544271, -9.39845200..."
1,3979151,5122,residential,Rua dos Pardais,None,B,0,0,F,F,377.9170,"LINESTRING (-9.395928700000001 38.9566218, -9...."
2,3979152,5113,primary,None,EN 247,B,50,0,F,F,223.9630,"LINESTRING (-9.419475500000001 38.9721483, -9...."
3,3979153,5122,residential,Travessa da Escola,None,B,0,0,F,F,70.1734,"LINESTRING (-9.3994056 38.9551873, -9.39906080..."
4,3979154,5122,residential,Rua de Baixo,None,B,0,0,F,F,242.9640,"LINESTRING (-9.3985764 38.9569177, -9.3985141 ..."
